<a href="https://colab.research.google.com/github/bene58/Detecta_BERT/blob/main/Detecta_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets scikit-learn torch pandas

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import numpy as np

In [ ]:
expanded_data = {
    "text": [
        "I hate you and hope you fail.",  # tóxico
        "You are an amazing person!",  # benigno
        "This is the worst thing ever, and I despise it.",  # tóxico
        "Thank you for your help, I appreciate it!",  # benigno
        "You are so stupid and useless!",  # tóxico
        "Have a great day ahead!",  # benigno
        "I can't stand your presence anymore.",  # tóxico
        "What a wonderful idea! I love it.",  # benigno
        "Get lost and never come back!",  # tóxico
        "This is a fantastic opportunity, go for it!",  # benigno
        "You are the reason for all the problems here!",  # tóxico
        "I believe in your potential and wish you success!",  # benigno
        "Why are you always so dumb?",  # tóxico
        "It's a pleasure to work with you.",  # benigno
        "Stop ruining everything, you're a failure!",  # tóxico
        "Thank you for being such a great friend.",  # benigno
        "Nobody likes you. Go away!",  # tóxico
        "Your work is excellent, keep it up!",  # benigno
        "I hope you never succeed in life.",  # tóxico
        "You're one of the best colleagues I've ever had!",  # benigno
        "Everything you do is a disaster!",  # tóxico
        "I am so proud of your achievements!",  # benigno
        "You're a complete waste of space.",  # tóxico
        "Keep shining, you're doing amazing!",  # benigno
        "I hate everything about you!",  # tóxico
        "Your kindness makes the world a better place.",  # benigno
        "You will never amount to anything!",  # tóxico
        "Your positive attitude inspires everyone around you.",  # benigno
        "You are a burden to everyone!",  # tóxico
        "Thanks for always being so supportive!",  # benigno
    ],
    "label": [
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0
    ],
}

# Cria um DataFrame e converte para o formato Dataset do Hugging Face
df = pd.DataFrame(expanded_data)
dataset = Dataset.from_pandas(df)

# Dividi o dataset em treino e teste
train_test_split = dataset.train_test_split(test_size=0.3)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [ ]:
# Carrega o tokenizador do BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

In [ ]:
# Carrega o modelo BERT para classificação com 2 classes
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Configura os parâmetros de treinamento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

import os
os.environ["WANDB_DISABLED"] = "true"


# Configura o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.707568
2,No log,0.664270
3,No log,0.561249
4,No log,0.521600
5,0.609100,0.504210
6,0.609100,0.486240
7,0.609100,0.475506
8,0.609100,0.466943
9,0.609100,0.461495
10,0.451300,0.459438


TrainOutput(global_step=20, training_loss=0.5302040815353394, metrics={'train_runtime': 31.6892, 'train_samples_per_second': 6.627, 'train_steps_per_second': 0.631, 'total_flos': 55253321625600.0, 'train_loss': 0.5302040815353394, 'epoch': 10.0})

In [ ]:
predictions = trainer.predict(tokenized_test)
preds = predictions.predictions.argmax(axis=1)

# Relatório de classificação
print(classification_report(test_dataset["label"], preds))

# Matriz de confusão
conf_matrix = confusion_matrix(test_dataset["label"], preds)
print("Matriz de Confusão:\n", conf_matrix)


              precision    recall  f1-score   support

           0       1.00      0.80      0.89         5
           1       0.80      1.00      0.89         4

    accuracy                           0.89         9
   macro avg       0.90      0.90      0.89         9
weighted avg       0.91      0.89      0.89         9

Matriz de Confusão:
 [[4 1]
 [0 4]]


In [ ]:
model.save_pretrained("./toxic-detector")
tokenizer.save_pretrained("./toxic-detector")


('./toxic-detector/tokenizer_config.json',
 './toxic-detector/special_tokens_map.json',
 './toxic-detector/vocab.txt',
 './toxic-detector/added_tokens.json')

In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification", model="./toxic-detector")

# Exemplos para testar
test_texts = [
    "You are horrible and no one likes you!",
    "You are doing a fantastic job, keep it up!",
    "I hope you fail miserably.",
    "I wish you a wonderful day!",
]

for text in test_texts:
    result = classifier(text)
    print(f"Texto: {text} => Resultado: {result}")

Device set to use cuda:0


Texto: You are horrible and no one likes you! => Resultado: [{'label': 'LABEL_1', 'score': 0.6073808670043945}]
Texto: You are doing a fantastic job, keep it up! => Resultado: [{'label': 'LABEL_0', 'score': 0.6876005530357361}]
Texto: I hope you fail miserably. => Resultado: [{'label': 'LABEL_1', 'score': 0.660823404788971}]
Texto: I wish you a wonderful day! => Resultado: [{'label': 'LABEL_0', 'score': 0.6819401979446411}]
